In [1]:
import pandas as pd 
import re
import datetime
import time
import random
from holistics import HolisticsAPI
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException

In [2]:
obj = HolisticsAPI(api_key = '53f+vEdkf1fUf/PUgXYd1RAHX+QPqQXY3LAvzT8eGpw=', url = 'https://secure.holistics.io')


In [3]:
# my_dataframe = obj.export_data(report_id='127694')

In [4]:
my_dataframe = pd.read_csv(r"C:\Users\TheLorry\Documents\Python Scripts\selenium dataset\deliveree\jobdetailfix.csv", dtype='unicode')

my_dataframe['pickup_datetime'] = pd.to_datetime(my_dataframe['pickup_datetime'])
my_dataframe['distance'] = pd.to_numeric(my_dataframe['distance'],errors='coerce').fillna(0)

my_dataframe.drop_duplicates(subset ="id", keep = "first", inplace=True)
mask = (my_dataframe['job_type'] =='b2c') 
my_dataframe = my_dataframe.loc[mask]
my_dataframe = my_dataframe.reset_index(drop=True)
mask = ((my_dataframe['pickup_datetime'] >= '2020-01-01') & (my_dataframe['pickup_datetime'] <= '2020-04-07'))
my_dataframe = my_dataframe.loc[mask]
my_dataframe = my_dataframe.reset_index(drop=True)
mask = (my_dataframe['country'] == 'id')
my_dataframe = my_dataframe.loc[mask]
my_dataframe = my_dataframe.reset_index(drop=True)

In [5]:
# my_dataframe
print(my_dataframe.pickup_lat[0])
print(my_dataframe.pickup_lng[0])
print(my_dataframe.full_address[0])

-6.8945198
107.5606479
Jalan Gunung Batu, Cigugur Tengah, Cimahi Tengah, Kota Cimahi, Jawa Barat, Indonesia


In [6]:
print(my_dataframe.drop_lat[0])
print(my_dataframe.drop_lng[0])
print(my_dataframe.drop_address[0])

-6.9173562
107.5536829
Jalan Rorojongrang, Melong, Kota Cimahi, Jawa Barat, Indonesia


In [7]:
vehicle_list = my_dataframe.groupby(['vehicle_name']).count()[['id']]
vehicle_list

,id
vehicle_name,
Double Bak,46
Double Box,33
Engkel Bak,687
Engkel Box,300
Pickup Bak,2520
Pickup Box,327
Van,216


In [20]:
# engkel box my_dataframe['vehicle_name'] == 'Engkel Box') | 
# van 
# mask = (my_dataframe['vehicle_name'] == 'Engkel Box')
# pickup | (my_dataframe['vehicle_name'] == 'Pickup Bak'
mask = (my_dataframe['vehicle_name'] == 'Pickup Bak')
item = my_dataframe.loc[mask]
item = item.reset_index(drop=True)
item = item[['id','drop_lng','drop_lat','drop_address','pickup_lat','pickup_lng','full_address','distance','total_price']]
item = item.sort_values(by=['distance'], ascending = False)
item

,id,drop_lng,drop_lat,drop_address,pickup_lat,pickup_lng,full_address,distance,total_price
1048,329697,112.6666413,-7.8714831,"Masjid Rukhul Islam, Song Song, Ardimulyo, Mal...",-6.2564104,106.8525585,"Apartemen kalibata city, Rawajati, South Jakar...",836.0,4215000.00
813,324006,112.3586174,-7.8835811,"Kantor Desa Pandansari Ngantang Malang, Jl. Be...",-6.9199668,107.6826944,"Jl. Cisaranten Baru I, Cisaranten Kulon, Bandu...",764.0,3048000.00
459,319906,104.6807584,-2.9359095,"Masjid Raya Citra Grand City, Jalan Boulevard ...",-6.9491711,107.6283793,"Soekarno-Hatta Street No.505, Cijagra, Bandung...",700.0,2359600.00
2060,346699,106.6388752,-6.1893896,"Gang M. Yamin 1, RT.004/RW.004, Babakan, Kota ...",-7.1853275,110.4390230,"Jl. Raya Ngempon, Klego, Ngempon, Kec. Bergas,...",490.0,2466000.00
2325,350133,106.8679899,-6.1826708,"Cempaka Putih, Kota Jakarta Pusat, Daerah Khus...",-7.0204858,110.4240158,"Jalan Dr. Wahidin, Jatingaleh, Kota Semarang, ...",437.0,2582000.00
...,...,...,...,...,...,...,...,...,...
1742,341143,106.9160197,-6.1533790,"Grand Emerald Apartmen, RT.13/RW.3, Pegangsaan...",-6.1536654,106.9167761,"Apartemen Gading Nias. Tower Alamanda, Jalan P...",1.0,324700.00
2052,346574,106.7435038,-6.2309926,"Gg. Susukan, RT.003/RW.009, Kreo, Kota Tangera...",-6.2328636,106.7434746,"Jl. H Muchtar Gg. Sawo 2 Kreo, Kota Tangerang,...",1.0,84700.00
1831,343397,106.8785859,-6.3167568,"Gang Jengki, RT.13/RW.10, Ciracas, East Jakart...",-6.3150509,106.8764959,"Jalan Pule Kavling 9, RT.2/RW.9, Ciracas, East...",0.0,244700.00
1532,338597,106.6660988,-6.1772290,"Graha Permata Poris, Jalan Panglima Polim, Nor...",-6.1782377,106.6668300,"Cluster Plawad Indah, RT.005/RW.005, North Por...",0.0,84700.00


In [ ]:
url_head  = "https://www.deliveree.com/id/en/intracity/?"
# url_vehicle = "calc=true&st_name=Engkel%20Box&"
url_vehicle = "calc=true&st_name=Pickup&"
url_pickup_dname = "dname=Indonesia&"
url_pickup_shortname = "shortname=Kota%20SBY&"
extracted_df = pd.DataFrame()
failelement = 0
for idx,each in item.iterrows():
    try:
        url_pickup_lat = f"lat={each.pickup_lat}&"
        url_pickup_lng = f"lng={each.pickup_lng}&"
        url_drop_dname = "dname=Timur,%20Indonesia&"
        url_drop_shortname = "shortname=Batu&"
        url_drop_lat = f"lat={each.drop_lat}&"
        url_drop_lng = f"lng={each.drop_lng}"

        url_maker = url_head+url_vehicle+url_pickup_dname+url_pickup_shortname+url_pickup_lat+url_pickup_lng+url_drop_dname+url_drop_shortname+url_drop_lat+url_drop_lng
        browser = webdriver.Chrome(executable_path=r"C:\WebDriver\bin\chromedriver.exe")
        browser.get(url_maker)
        time.sleep(random.randint(8,10))
#         WebDriverWait(browser,20).until(EC.presence_of_element_located((By.XPATH,"""//*[@id='fare-estimator']/div[1]/div/div[1]/div[1]/div[2]/b"""))).click()
        element_price = browser.find_element_by_xpath("""//*[@id='fare-estimator']/div[1]/div/div[1]/div[1]/div[2]/b""")
        price = element_price.text
        browser.close()
        newitem = pd.DataFrame(data={"pickup_lat":[each.pickup_lat], "pickup_lng":[each.pickup_lng], "full_address":each.full_address,
                                    "drop_lat":[each.drop_lat], "drop_lng":[each.drop_lng], "drop_address":each.drop_address,
                                    "distance":[each.distance], "thelorry_price":[each.total_price], "deliveree_price" : [price]})
        extracted_df = extracted_df.append(newitem)
        extracted_df = extracted_df.reset_index(drop=True)
    except NoSuchElementException as exception:
            failelement = failelement + 1
            browser.close()
            continue
    except:
            extracted_df.to_csv (r'C:\Users\TheLorry\Documents\Python Scripts\selenium dataset\deliveree\result3csv', index = False, header=True)
extracted_df.to_csv (r'C:\Users\TheLorry\Documents\Python Scripts\selenium dataset\deliveree\result3.csv', index = False, header=True)

In [ ]:
price

In [12]:
extracted_df.to_csv (r'C:\Users\TheLorry\Documents\Python Scripts\selenium dataset\deliveree\result3.csv', index = False, header=True)

In [ ]:
newitem